In [4]:
%matplotlib inline
import os
import numpy as np

from random import randint
from threading import Thread
from time import sleep, time

from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from tqdm.notebook import tqdm

from gwpy.timeseries import TimeSeries
from ts_outlier_detection import *
from ts_outlier_detection.plotting import *
from utils import *

time_header = 'GPStime'
fast_scattering = 'Fast Scattering'
koi_fish = 'Koi Fish'
tomte = 'Tomte'
repeating_blips = 'Repeating Blips'
scattered_light = 'Scattered Light'

CACHE = '/mnt/e/datacache/'

def load_cached_data(target, pathname, start):
    name = os.path.basename(pathname)
    name = os.path.splitext(name)[0]
    det, gps_time = name.split('_')
    gps_time = float(gps_time)
    target.append((TimeSeries.read(pathname), det, gps_time))
    print(f'Fetched {len(target)} events after {round(time()-start, 2)} seconds')

def get_cached_data_only(name):
    cache_path = os.path.join(CACHE, name)
    files = os.listdir(cache_path)
    glitch_ts = []
    
    start = time()
    threads = []
    for fi in files:
        full_path = os.path.join(cache_path, fi)
        if os.path.isfile(full_path):
            threads.append(Thread(target=load_cached_data, args=(glitch_ts, full_path, start)))
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    
    return glitch_ts

In [5]:
## Gradient descent

def gradient_descent(
    params, learning_rate, training_set, loss_function,
    downsample=1, epochs=3, batch_size=5, max_iter=100, seed=42, threshold=1e-4
):
    rng = np.random.default_rng(seed)
    for epoch in range(epochs):
        print(f'Starting training epoch {epoch+1}/{epochs}')
        rng.shuffle(training_set)
        avg_losses = []

        for i in tqdm(range(0, len(training_set), batch_size)):
            for _ in range(max_iter):
                grad = np.zeros(2)
                batch_loss = 0

                for ts, _, actual in training_set[i:i+batch_size]:
                    data = ts.value[::downsample]
                    times = ts.times.value[::downsample]

                    def loss(n, e):
                        ctof = TemporalOutlierFactor(n_neighbors=n, event_length=e)
                        ctof.fit(data, times)
                        return loss_function(actual, times[ctof.get_outlier_indices()])

                    batch_loss += loss(*params)
                    grad += estimate_gradient(loss, params, eps=1)

                grad /= batch_size
                avg_losses.append(batch_loss/batch_size)
                if np.max(np.abs(grad)) < threshold:
                    break
                params -= np.around(learning_rate * ((grad > 0).astype(int) - (grad < 0).astype(int))).astype(int)
                for i in range(params.size):
                    if params[i] < 2:
                        params[i] = 2
    
        fig, ax = plt.subplots(1, 1, figsize=(10,6))
        ax.set_title(f'Epoch {epoch+1} losses')
        ax.plot(np.arange(len(avg_losses)), avg_losses, 'k.')
        ax.set_xlabel('Batch number')
        ax.set_ylabel('Average loss over batch')
        ax.grid(True)
        print(f'Current optimal parameters: {params}')
        
    return params

## Grid search

def gen_filename(det, gps_time):
    return f'{det}_{gps_time}'

def tof_grid_search(k_range, m_range, hdatas, loss_function=stdev_loss, save_loc=None, aggregate=None):
    loss_per_event = []
    for hdata, det, gps_time in tqdm(hdatas, desc='Training Samples'):
        fname = gen_filename(det, gps_time)
        cached = False
        
        if save_loc:
            full_path = os.path.join(save_loc, fname)
            csv_path = full_path+'.csv'
            try:
                loss_per_event.append(np.loadtxt(csv_path, delimiter=','))
                cached = True
            except Exception as e:
                print(f'Error fetching cached data at {csv_path}: {e}')
            
        if not cached:
            data = hdata.value
            times = hdata.times.value
            loss_map = np.zeros((len(k_range), len(m_range)))
            for i, n_neighbors in enumerate(k_range):
                for j, event_length in enumerate(m_range):
                    ctof = TemporalOutlierFactor(n_neighbors=n_neighbors, event_length=event_length)
                    ctof.fit(data, times)
                    loss_map[i, j] = loss_function(gps_time, times[ctof.get_outlier_indices()])
            loss_per_event.append(loss_map)
        
        if save_loc and not cached:
            plot_grid_search(k_range, m_range, loss_map, os.path.join(save_loc, fname+'.png'))
            np.savetxt(csv_path, loss_map, delimiter=',')
            
    if len(loss_per_event) > 1 and aggregate is not None:
        aggregated_loss = aggregate(loss_per_event)
        if save_loc:
            full_path = os.path.join(save_loc, f'aggregate_using_{aggregate.__name__}')
            csv_path = full_path+'.csv'
            plot_grid_search(k_range, m_range, aggregated_loss, full_path+'_imshow.png', style='imshow')
            plot_grid_search(
                k_range, m_range, aggregated_loss, full_path+'_contour.png',
                style='contour', kwargs={'levels': 15}
            )
            if os.path.isfile(csv_path):
                print(f'Warning: {csv_path} already exists, overwriting...')
            np.savetxt(csv_path, aggregated_loss, delimiter=',')
            
        return aggregated_loss
    
    return loss_per_event

def plot_grid_search(k_range, m_range, loss_map, fname, style='imshow', kwargs={}):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    k0 = k_range[0]
    k1 = k_range[-1]
    m0 = m_range[0]
    m1 = m_range[-1]
    getattr(ax, style)(loss_map, extent=[k0, k1, m0, m1], aspect=(k1-k0)/(m1-m0), **kwargs)
    ax.set_xlabel('n_neighbors')
    ax.set_ylabel('event_length')
    ax.grid(False)
    cbar = ax.colorbar()
    cbar.set_label('Loss (lower is better)')
    fig.savefig(fname)
    plt.close(fig)

## Training Grid Search

In [6]:
k_range = list(range(2, 41))
m_range = list(range(300, 2001, 50))

def sum_loss_normalized(arr):
    prev = arr[0]
    prev /= np.abs(np.min(prev))
    for curr in arr[1:]:
        prev -= curr / np.min(curr)
    prev /= np.abs(np.min(prev))
    return prev

loss_per_event = tof_grid_search(
    k_range, m_range, tomte_ts,
    save_loc='data/tomte_grid_search', aggregate=sum_loss_normalized
)

NameError: name 'tomte_ts' is not defined

In [3]:
to_sum = [
    'data/koi_fish_grid_search/aggregate_using_sum_loss_normalized.csv',
    'data/repeating_blips_grid_search/aggregate_using_sum_loss_normalized.csv',
    'data/tomte_grid_search/aggregate_using_sum_loss_normalized.csv'
]

shape = (39, 35)
canvas = np.zeros(shape)
for fi in to_sum:
    loss_array = np.loadtxt(fi, delimiter=',')[:shape[0], :shape[1]]
    canvas += loss_array

canvas /= np.max(np.abs(canvas))
plot_grid_search(k_range, m_range, canvas, 'data/sum_loss_over_glitch_types.png', style='imshow')

NameError: name 'plot_grid_search' is not defined

## Let's see what the loss function actually looks like...

In [14]:
N_NEIGHBORS = []
LOSSES = []
PLOTS = []

hdata, _, gps_time = training_set[0]
data = hdata.value
times = hdata.times.value

fig = plt.figure(figsize=(20,6), constrained_layout=True)
gs = fig.add_gridspec(2, 2)
rect1 = fig.add_subplot(gs[0,0])
rect1.set_ylabel('Strain')
rect2 = fig.add_subplot(gs[1,0], sharex=rect1)
rect2.set_ylabel('TOF')
rect2.set_xlabel('GPS Time (s)')
rect3 = fig.add_subplot(gs[:,1])
rect3.set_ylabel('Loss')
rect3.set_xlabel('n_neighbors')

def animate(frame):
    global PLOTS
    ctof = TemporalOutlierFactor(n_neighbors=frame, event_length=1024)
    ctof.fit(data, times)
    N_NEIGHBORS.append(frame)
    LOSSES.append(stdev_loss(gps_time, times[ctof.get_outlier_indices()]))
    
    for plot in PLOTS: plot.remove()
    PLOTS = plot_ts_outliers(ctof, [rect1, rect2])
    PLOTS.append(rect3.plot(N_NEIGHBORS, LOSSES, 'k.-')[0])
    return PLOTS

n_range = tqdm(list(range(2, 31)))
anim = animation.FuncAnimation(
    fig, animate, frames=n_range, interval=200, blit=True)
plt.close()
HTML(anim.to_html5_video())

  0%|          | 0/29 [00:00<?, ?it/s]

# Koi Fish

In [6]:
## Koi Fish

koi_fish_ts = get_cached_data_only(koi_fish)
    
rng = np.random.default_rng(3)
rng.shuffle(koi_fish_ts)
training_size = 0.8
split_idx = int(training_size*len(koi_fish_ts))
training_set = koi_fish_ts[:split_idx]
test_set = koi_fish_ts[split_idx:]

Fetched 1 events after 3.96 secondsFetched 2 events after 4.17 seconds

Fetched 3 events after 4.56 secondsFetched 4 events after 4.67 seconds
Fetched 5 events after 4.71 seconds

Fetched 6 events after 4.91 secondsFetched 7 events after 4.93 seconds

Fetched 8 events after 5.0 secondsFetched 9 events after 5.01 seconds

Fetched 10 events after 5.06 secondsFetched 11 events after 5.07 secondsFetched 12 events after 5.08 seconds

Fetched 13 events after 5.11 seconds
Fetched 14 events after 5.15 seconds
Fetched 15 events after 5.2 seconds
Fetched 16 events after 5.23 seconds

Fetched 17 events after 5.25 secondsFetched 18 events after 5.28 seconds
Fetched 19 events after 5.29 secondsFetched 20 events after 5.3 seconds
Fetched 21 events after 5.31 seconds
Fetched 22 events after 5.31 seconds
Fetched 23 events after 5.31 seconds

Fetched 24 events after 5.34 seconds

Fetched 25 events after 5.36 secondsFetched 26 events after 5.37 seconds
Fetched 27 events after 5.38 secondsFetched 28 even

/home/azimuth/.local/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## Training Gradient Descent

In [ ]:
## Parameters to optimize (n_neighbors, event_length)
params = np.array([4, 1024]) # initial values
learning_rate = np.array([1, 5])

## Gradient descent
best_k, best_length = gradient_descent(
    params, learning_rate, training_set, inverse_diff_loss,
    epochs=5, batch_size=1, max_iter=100, seed=42
)

## Testing

In [ ]:
## Testing
best_k = 4
event_length = 520
optimal_tof = TemporalOutlierFactor(n_neighbors=best_k, event_length=best_length)
losses = []

for ts, _, actual in tqdm(test_set):
    data = ts.value
    times = ts.times.value
    optimal_tof.fit(data, times)
    losses.append(diff_loss(actual, times[optimal_tof.get_outlier_indices()]))
    
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.hist(losses, )
ax.set_title('Koi Fish Test Set Losses')
ax.set_xlabel('Loss')
ax.set_ylabel('Events')
ax.grid(True)

## Plot representative examples
for hdata, det, gps_time in tqdm(rng.choice(test_set, size=3, replace=False)):
    rdata = get_raw_event(det, gps_time, length=2, offset=0, edges=10)
    plot_outlier_comparison(hdata, rdata, n_neighbors=best_k, event_length=best_length)

# Fast Scattering

In [ ]:
## Fast Scattering

fast_scattering_ts = get_cached_data_only(fast_scattering)

rng = np.random.default_rng(3)
rng.shuffle(fast_scattering_ts)
training_size = 0.8
split_idx = int(training_size*len(fast_scattering_ts))
training_set = fast_scattering_ts[:split_idx]
test_set = fast_scattering_ts[split_idx:]

## Let's see the loss function

In [ ]:
N_NEIGHBORS = []
LOSSES = []
PLOTS = []

hdata, _, gps_time = training_set[4]
data = hdata.value
times = hdata.times.value

fig = plt.figure(figsize=(20,6), constrained_layout=True)
gs = fig.add_gridspec(2, 2)
rect1 = fig.add_subplot(gs[0,0])
rect1.set_ylabel('Strain')
rect2 = fig.add_subplot(gs[1,0], sharex=rect1)
rect2.set_ylabel('TOF')
rect2.set_xlabel('GPS Time (s)')
rect3 = fig.add_subplot(gs[:,1])
rect3.set_ylabel('Loss')
rect3.set_xlabel('n_neighbors')

def animate(frame):
    global PLOTS
    ctof = TemporalOutlierFactor(n_neighbors=frame, event_length=2048)
    ctof.fit(data, times)
    N_NEIGHBORS.append(frame)
    LOSSES.append(stdev_loss(gps_time, times[ctof.get_outlier_indices()]))
    
    for plot in PLOTS: plot.remove()
    PLOTS = plot_ts_outliers(ctof, [rect1, rect2])
    PLOTS.append(rect3.plot(N_NEIGHBORS, LOSSES, 'k.-')[0])
    return PLOTS

n_range = tqdm(list(range(2, 41)))
anim = animation.FuncAnimation(
    fig, animate, frames=n_range, interval=500, blit=True)
plt.close()
HTML(anim.to_html5_video())

## Training

In [ ]:
## Parameters to optimize (n_neighbors, event_length)
params = np.array([4, 1024]) # initial values
learning_rate = np.array([1, 30])

## Gradient descent
best_k, best_length = gradient_descent(
    params, learning_rate, training_set, diff_loss,
    downsample=4, epochs=3, batch_size=5, max_iter=100, seed=42
)

In [ ]:
best_k = 5
best_length = 2048

## Testing
optimal_tof = TemporalOutlierFactor(n_neighbors=best_k, event_length=best_length)
losses = []

for ts, _, actual in tqdm(test_set):
    data = ts.value
    times = ts.times.value
    optimal_tof.fit(data, times)
    losses.append(diff_loss(actual, times[optimal_tof.get_outlier_indices()]))
    
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.hist(losses)
ax.set_title('Fast Scattering Test Set Losses')
ax.set_xlabel('Loss')
ax.set_ylabel('Events')
ax.grid(True)

## Plot representative examples
for hdata, det, gps_time in tqdm(rng.choice(test_set, size=3, replace=False)):
    rdata = get_raw_event(det, gps_time, length=2, offset=0, edges=10)
    plot_outlier_comparison(hdata, rdata, n_neighbors=best_k, event_length=best_length)

# Scattered Light

In [ ]:
## Scattered Light

scattered_light_ts = get_cached_data_only(scattered_light)

rng = np.random.default_rng(42)
rng.shuffle(scattered_light_ts)
training_size = 0.8
split_idx = int(training_size*len(scattered_light_ts))
training_set = scattered_light_ts[:split_idx]
test_set = scattered_light_ts[split_idx:]

In [ ]:
N_NEIGHBORS = []
LOSSES = []
PLOTS = []

hdata, _, gps_time = training_set[4]
# hdata = hdata.bandpass(20, 100)
data = hdata.value
times = hdata.times.value

fig = plt.figure(figsize=(20,6), constrained_layout=True)
gs = fig.add_gridspec(2, 2)
rect1 = fig.add_subplot(gs[0,0])
rect1.set_ylabel('Strain')
rect2 = fig.add_subplot(gs[1,0], sharex=rect1)
rect2.set_ylabel('TOF')
rect2.set_xlabel('GPS Time (s)')
rect3 = fig.add_subplot(gs[:,1])
rect3.set_ylabel('Loss')
rect3.set_xlabel('n_neighbors')

def animate(frame):
    global PLOTS
    ctof = TemporalOutlierFactor(n_neighbors=frame, event_length=30*2048)
    ctof.fit(data, times)
    N_NEIGHBORS.append(frame)
    LOSSES.append(stdev_loss(gps_time, times[ctof.get_outlier_indices()]))
    
    for plot in PLOTS: plot.remove()
    PLOTS = plot_ts_outliers(ctof, [rect1, rect2])
    PLOTS.append(rect3.plot(N_NEIGHBORS, LOSSES, 'k.-')[0])
    return PLOTS

n_range = tqdm(list(range(2, 41)))
anim = animation.FuncAnimation(
    fig, animate, frames=n_range, interval=500, blit=True)
plt.close()
HTML(anim.to_html5_video())

In [ ]:
## Parameters to optimize (dims, n_neighbors, event_length)
params = np.array([100, 100, 2048]) # initial values
learning_rate = np.array([3, 3, 30])

## Gradient descent
best_dims, best_k, best_length = gradient_descent(
    params, learning_rate, training_set, diff_loss,
    epochs=5, batch_size=5, max_iter=100, seed=42
)

In [ ]:
best_k = 20
best_length = 4*2048
## Testing
optimal_tof = TemporalOutlierFactor(n_neighbors=best_k, event_length=best_length)
losses = []

for ts, _, actual in tqdm(test_set):
    data = ts.value
    times = ts.times.value
    optimal_tof.fit(data, times)
    losses.append(stdev_loss(actual, times[optimal_tof.get_outlier_indices()]))
    
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.hist(losses)
ax.set_title('Scattered Light Test Set Losses')
ax.set_xlabel('Loss')
ax.set_ylabel('Events')
ax.grid(True)

## Plot representative examples
for hdata, det, gps_time in tqdm(rng.choice(test_set, size=5, replace=False)):
    rdata = get_raw_event(det, gps_time, length=150, offset=0, edges=10)
    plot_outlier_comparison(hdata, rdata, n_neighbors=best_k, event_length=best_length)

# Tomte

In [7]:
tomte_ts = get_cached_data_only(tomte)
    
rng = np.random.default_rng(3)
rng.shuffle(tomte_ts)
training_size = 0.8
split_idx = int(training_size*len(tomte_ts))
training_set = tomte_ts[:split_idx]
test_set = tomte_ts[split_idx:]

Fetched 1 events after 11.3 secondsFetched 2 events after 11.32 secondsFetched 3 events after 11.33 seconds
Fetched 4 events after 11.33 seconds


Fetched 5 events after 11.37 secondsFetched 6 events after 11.37 seconds

Fetched 7 events after 11.38 secondsFetched 8 events after 11.39 seconds
Fetched 9 events after 11.42 secondsFetched 10 events after 11.42 seconds

Fetched 11 events after 11.43 seconds
Fetched 12 events after 11.44 seconds

Fetched 13 events after 11.47 seconds
Fetched 14 events after 11.48 seconds
Fetched 15 events after 11.53 seconds
Fetched 16 events after 11.55 secondsFetched 17 events after 11.56 seconds

Fetched 18 events after 11.62 seconds
Fetched 19 events after 11.69 secondsFetched 20 events after 11.7 secondsFetched 21 events after 11.71 secondsFetched 22 events after 11.72 seconds



Fetched 23 events after 11.74 secondsFetched 24 events after 11.78 seconds
Fetched 25 events after 11.82 seconds
Fetched 26 events after 11.82 seconds

Fetched 27 events after

/home/azimuth/.local/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


# Repeating Blips

In [12]:
repeating_blips_ts = get_cached_data_only(repeating_blips)
    
rng = np.random.default_rng(3)
rng.shuffle(repeating_blips_ts)
training_size = 0.8
split_idx = int(training_size*len(repeating_blips_ts))
training_set = repeating_blips_ts[:split_idx]
test_set = repeating_blips_ts[split_idx:]

Fetched 1 events after 0.62 seconds
Fetched 2 events after 0.74 seconds
Fetched 3 events after 0.86 secondsFetched 4 events after 0.87 seconds

Fetched 5 events after 0.89 seconds
Fetched 6 events after 0.94 seconds
Fetched 7 events after 1.03 seconds
Fetched 8 events after 1.06 secondsFetched 9 events after 1.08 seconds

Fetched 10 events after 1.19 seconds
Fetched 11 events after 1.23 seconds
Fetched 12 events after 1.24 secondsFetched 13 events after 1.25 seconds

Fetched 14 events after 1.4 seconds
Fetched 15 events after 1.44 seconds
Fetched 16 events after 1.44 seconds
Fetched 17 events after 1.6 secondsFetched 18 events after 1.67 secondsFetched 19 events after 1.71 seconds

Fetched 20 events after 1.74 seconds

Fetched 21 events after 1.77 seconds
Fetched 22 events after 1.85 seconds
Fetched 23 events after 1.96 secondsFetched 24 events after 1.97 seconds
Fetched 25 events after 1.98 secondsFetched 26 events after 2.01 seconds


Fetched 27 events after 2.03 secondsFetched 28 ev

/home/azimuth/.local/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
